In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have a method to fetch data from Yahoo Finance API
# df = fetch_data_from_yahoo()

# Use 'Close' prices for prediction
data = df['Close'].values.reshape(-1, 1)

# Normalize data
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# Prepare training data with a window of 60 days
X, y = [], []
for i in range(60, len(normalized_data) - 1):
    X.append(normalized_data[i-60:i, 0])
    y.append(normalized_data[i, 0])
X, y = np.array(X), np.array(y)

# Train-test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Reshape for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build an advanced LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=100, return_sequences=True), input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=100))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Use early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=64, callbacks=[early_stopping])

# Predict and compute RMSE
predicted_prices = model.predict(X_test)
predicted_prices = scaler.inverse_transform(predicted_prices)
rmse = np.sqrt(mean_squared_error(y_test, predicted_prices))
print("RMSE: ", rmse)


